# Users motivations

Finally, to give some insights about the why question, I did a manual analysis of the top 12 most contributors as well as people that have more weaponising edits than not weaponising edits. The manual analysis is based on the User page and the User talk page mainly. We are seeking banners, templates and semantic behaviour that tells a lot about someone’s identity, ideology and culture. However, those features can give some insights and some potential 
direction but we cannot and we won’t determine someone's culture only based on his wikipedia page. 


In [2]:
import requests
import mwparserfromhell
import csv
import json
import time
from tqdm import tqdm
import pandas as pd

In [5]:
df = pd.read_csv('../../datas/final/small_db_preprocess.csv')

## Comments analysis

In the exploring notebook, we saw that IP users are more often editing without leaving any comment than registered users. We also noticed that, in proportion, editing classified as *not weaponized*, are often more posted without any comments than for *weaponized edits*, which is at first a bit conter intuitive and demonstrates how weaponization can be complex and rich. Let's then raise the following research questions :
* Is there any difference between comments in *weaponized* vs *not weaponized* edits ?

In [20]:
df_comment_weap = df[
    (df['weaponised'] == 'Weaponised') &
    (df['comment'].notna())
]
df_comment_no_weap = df[
    (df['weaponised'] == 'Not Weaponised') &
    (df['comment'].notna())
]

def comment_lenght(dataframe):
    

,article,user,date,comment,llm_output,weaponised,year,user_type
8,History of Ukraine,Irpen,2006-06-14T15:11:52Z,"read Holodomor, numbers are discussed",The change made in this revision is the additi...,Weaponised,2006,Registered
9,History of Ukraine,141.149.187.165,2006-08-22T20:44:34Z,/* Ukraine in World War II */,The change made in this revision is the additi...,Weaponised,2006,Anonymous (IP)
25,COVID-19 pandemic in Ukraine,Perep80,2020-03-29T00:27:43Z,Updates to table of daily cases,"Described change: Added new columns for ""Death...",Weaponised,2020,Registered
51,COVID-19 pandemic in Ukraine,UP9,2020-04-07T16:56:45Z,Added 'so-called' and 'annexed',The change made is the addition of the phrases...,Weaponised,2020,Registered
60,COVID-19 pandemic in Ukraine,Adûnâi,2020-04-10T03:26:29Z,"Wording corrected, grammar corrected (definite...",The change made was a rephrasing of terms rela...,Weaponised,2020,Registered
...,...,...,...,...,...,...,...,...
6882,2014 pro-Russian unrest in Ukraine,SovietKomarov,2014-04-18T02:59:08Z,/* Largest protests by date and attendance */,The change made is the addition of several ent...,Weaponised,2014,Registered
6884,2014 pro-Russian unrest in Ukraine,RGloucester,2014-04-18T15:08:39Z,/* Government building seizures */ link,"Change: Rephrased the phrase ""sent soldiers aw...",Weaponised,2014,Registered
6913,History of Ukraine,Wiki13,2012-01-28T13:04:04Z,Undid revision 473680318 by [[Special:Contribu...,The change made was the addition of several im...,Weaponised,2012,Registered
6915,COVID-19 pandemic in Ukraine,LSGH,2021-04-28T05:29:44Z,Updating number of cases in infobox,"Change: Updated the number of confirmed cases,...",Weaponised,2021,Registered


## Templates analysis

This section is an analysis into the users template banner used. Templates are very often used by user as : 
* identity markers
* ideological broadcasting
* group affiliations signals
* defensive framings

In [ ]:
URL = "https://en.wikipedia.org/w/api.php"
HEADERS = {
    "User-Agent": "DH_Project/1.0 (https://www.epfl.ch/labs/dhlab/; maxime.garambois@epfl.ch)"
}

SLEEP_TIME = 0.5  # be polite to the API


def fetch_wikitext(title):
    params = {
        "action": "query",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": title,
        "format": "json",
        "formatversion": "2"
    }

    r = requests.get(URL, params=params, headers=HEADERS)
    r.raise_for_status()
    data = r.json()

    pages = data.get("query", {}).get("pages", [])
    if not pages or "missing" in pages[0]:
        return None

    revisions = pages[0].get("revisions", [])
    if not revisions:
        return None

    return revisions[0]["slots"]["main"]["content"]


def extract_templates(wikitext):
    wikicode = mwparserfromhell.parse(wikitext)
    templates = []

    for tpl in wikicode.filter_templates(recursive=True):
        templates.append({
            "template_name": str(tpl.name).strip(),
            "parameters": {
                str(param.name).strip(): str(param.value).strip()
                for param in tpl.params
            }
        })

    return templates


def process_user(username):
    results = []

    for page_type, title in [
        ("user", f"User:{username}"),
        ("user_talk", f"User talk:{username}")
    ]:
        try:
            wikitext = fetch_wikitext(title)
            if not wikitext:
                continue

            templates = extract_templates(wikitext)

            for tpl in templates:
                results.append({
                    "username": username,
                    "page_type": page_type,
                    "template_name": tpl["template_name"],
                    "parameters": tpl["parameters"]
                })

        except Exception as e:
            print(f"[ERROR] {username} ({page_type}): {e}")

        time.sleep(SLEEP_TIME)

    return results


def run():
    all_results = []

    for user in tqdm(top10_users, desc="Processing users"):
        all_results.extend(process_user(user))

    # Save JSON (full fidelity)
    with open("user_templates_raw.json", "w", encoding="utf-8") as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    # Save CSV (easy inspection)
    with open("user_templates_raw.csv", "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["username", "page_type", "template_name", "parameters"]
        )
        writer.writeheader()
        for row in all_results:
            writer.writerow(row)

    print("✔ Extraction complete")

In [6]:
df_templates = pd.read_csv('user_templates_raw.csv')
df_templates

FileNotFoundError: [Errno 2] No such file or directory: 'user_templates_raw.csv'